In [1]:
from flashtext import KeywordProcessor
import pandas as pd
import os
import gensim
import logging
import datetime

logging.getLogger().setLevel(logging.INFO)

from gensim.models import Word2Vec, KeyedVectors

In [2]:
root_dir = os.path.abspath(os.pardir)
data_dir = os.path.join(root_dir, "data")
model_dir = os.path.join(root_dir, "model")
tag_table_file = os.path.join(data_dir, "paper_tag_table.csv")
paper_table_file = os.path.join(data_dir, "paper_table.csv")
pre_trained_model = os.path.join(model_dir, "GoogleNews-vectors-negative300.bin")
output_model_file = os.path.join(model_dir, "model.bin")

In [3]:
tag_set = set(pd.read_csv(tag_table_file)['tag_text'].dropna().tolist())
abstract_list = pd.read_csv(paper_table_file)['abstract'].dropna().tolist()[:2]

In [4]:
from phrase_word2vec import PhraseWord2VEC

In [5]:
obj = PhraseWord2VEC()
obj.train(tag_set, abstract_list, pre_trained_model, output_model_file)

INFO:root:pre processing input sentences 
INFO:gensim.models.utils_any2vec:loading projection weights from c:\Users\saurabh\Documents\saurabh\ms\nus\ra\phrase_word2vec\model\GoogleNews-vectors-negative300.bin
DEBUG:smart_open.smart_open_lib:{'uri': 'c:\\Users\\saurabh\\Documents\\saurabh\\ms\\nus\\ra\\phrase_word2vec\\model\\GoogleNews-vectors-negative300.bin', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'ignore_ext': False, 'transport_params': None}
INFO:gensim.models.utils_any2vec:loaded (3000000, 300) matrix from c:\Users\saurabh\Documents\saurabh\ms\nus\ra\phrase_word2vec\model\GoogleNews-vectors-negative300.bin
INFO:root:starting training...
INFO:gensim.models.base_any2vec:training model with 3 workers on 3000000 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
DEBUG:gensim.models.base_any2vec:job loop exiting, total 1 jobs
Exception in thread Thread-6:
Traceback (most recent call l

In [ ]:
Word2